In [30]:
import numpy as np 
import pandas as pd 

from sklearn.metrics import accuracy_score
from sklearn import metrics

# importing models
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from xgboost import XGBClassifier

from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

from sklearn.model_selection import train_test_split # Import train_test_split function

from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation

from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score

Read the train and test datasets

In [31]:
train=pd.read_csv("train_I.csv")
test=pd.read_csv("test_I.csv")

check the count of null values in each column

In [32]:
null_counts = train.isnull().sum()
# Filter for columns with non-zero null counts
null_cols = null_counts[null_counts > 0]
print(null_cols)

MaxPartialCharge        420
MinPartialCharge        420
MaxAbsPartialCharge     420
MinAbsPartialCharge     420
BCUT2D_MWHI            3813
BCUT2D_MWLOW           3813
BCUT2D_CHGHI           3813
BCUT2D_CHGLO           3813
BCUT2D_LOGPHI          3813
BCUT2D_LOGPLOW         3813
BCUT2D_MRHI            3813
BCUT2D_MRLOW           3813
dtype: int64


Fill the null values with mean value

In [33]:
train =train.fillna(train.mean())
test=test.fillna(train.mean())

/var/folders/cz/r8b553t535v1b2qw2sv9kd3c0000gn/T/ipykernel_4894/705866291.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  train =train.fillna(train.mean())
/var/folders/cz/r8b553t535v1b2qw2sv9kd3c0000gn/T/ipykernel_4894/705866291.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  test=test.fillna(train.mean())


In [34]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75377 entries, 0 to 75376
Columns: 213 entries, MaxEStateIndex to Expected
dtypes: float64(104), int64(106), object(3)
memory usage: 122.5+ MB


Drop the string columns from train and test dataset

In [35]:
train.drop(["Id1","Smiles","Id"],axis=1,inplace=True)
test.drop(["Id1","Smiles"],axis=1,inplace=True)

In [36]:
train

,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,...,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea,Id2,Expected
0,9.316200,-1.533785,9.316200,0.150485,0.794714,317.599,306.511,315.982463,100,0,...,0,0,0,0,0,0,0,0,1644,2
1,10.532611,0.333788,10.532611,0.333788,0.516641,156.269,136.109,156.151415,66,0,...,0,0,0,0,0,0,4,0,2451,2
2,2.433032,0.000000,2.433032,0.000000,0.251327,362.086,313.702,361.347528,148,0,...,0,0,0,0,0,0,12,0,1384,2
3,10.355080,-0.613825,10.355080,0.282361,0.487998,255.665,245.585,255.052302,90,0,...,0,0,0,0,0,0,0,0,16,2
4,0.000000,0.000000,0.000000,0.000000,0.237972,149.894,149.894,149.894242,8,0,...,0,0,0,0,0,0,0,0,1856,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75372,11.460021,-3.868472,11.460021,0.053611,0.712426,230.245,220.165,230.036128,82,0,...,0,0,0,0,0,0,0,0,33,2
75373,5.928972,-2.841623,5.928972,0.082346,0.720533,313.747,296.611,313.041677,104,0,...,0,0,0,0,0,0,0,0,1632,1
75374,4.975926,0.848333,4.975926,0.848333,0.596343,167.258,162.218,166.986341,50,0,...,0,0,0,1,0,0,0,0,1373,1
75375,10.241948,0.324028,10.241948,0.324028,0.519485,128.215,112.087,128.120115,54,0,...,0,0,0,0,0,0,0,0,2,2


get the columns with high correlation

In [37]:
corr = train.corr()
cols = []
for i in range(len(corr.columns)):
    for j in range(i):
        if abs(corr.iloc[i, j]) > 0.8:
            cols.append((corr.columns[i], corr.columns[j]))
col1 = [x[0] for x in cols ]
col2 = [x[1] for x in cols ]
feature1=list(set(col1))
feature2=list(set(col2))
#feature2.append("MinPartialCharge")

Drop the columns having high correlation

In [38]:
train.drop(feature2,axis=1,inplace=True)
test. drop(feature2,axis=1,inplace=True)

In [39]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75377 entries, 0 to 75376
Columns: 148 entries, MaxAbsEStateIndex to Expected
dtypes: float64(65), int64(83)
memory usage: 85.1 MB


Create X by dropping the expected column and y with expected column to train and predict the model

In [40]:
X=train.drop(["Expected","Id2"],axis=1)
y=train["Expected"]

Used GridsearchCV to get the best number of features to provide as input to Recursive feature elimination method.

from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import RFE
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# X is the feature matrix and y is the target variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a decision tree classifier
clf = DecisionTreeClassifier()

# Create a recursive feature elimination object
rfe = RFE(estimator=clf, step=1)

# Set up the parameter grid
param_grid = {'n_features_to_select': [30,40,50,60,70]}

# Perform a grid search over the parameter grid
grid_search = GridSearchCV(estimator=rfe, param_grid=param_grid, cv=5, scoring='accuracy')

# Fit the grid search object to the training data
grid_search.fit(X_train, y_train)

# Extract the best estimator and the optimal value of n_features_to_select
best_estimator = grid_search.best_estimator_
optimal_n_features = grid_search.best_params_['n_features_to_select']

# Fit the best estimator to the training data
best_estimator.fit(X_train, y_train)

# Make predictions on the test data using the selected features
y_pred = best_estimator.predict(X_test)

# Evaluate the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


selected 60 columns using Recursive feature elimination. The parameter n_features_to_select for RFE is selected based on the output from gridsearchCV

from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# X is the feature matrix and y is the target variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a decision tree classifier
clf = DecisionTreeClassifier()

# Create a recursive feature elimination object
rfe = RFE(estimator=clf, n_features_to_select=60, step=1)

# Fit the recursive feature elimination object to the training data
rfe.fit(X_train, y_train)

# Extract the selected features
selected_features = X_train.columns[rfe.support_]

# Fit the decision tree classifier to the selected features
clf.fit(X_train[selected_features], y_train)

# Make predictions on the test data using the selected features
y_pred = clf.predict(X_test[selected_features])

# Evaluate the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


select the features received from Recursive feature elimination method and set X and y values

In [41]:
selected_features =['MaxAbsEStateIndex', 'MinAbsEStateIndex', 'qed', 'MinPartialCharge',
       'MaxAbsPartialCharge', 'MinAbsPartialCharge', 'FpDensityMorgan3',
       'BCUT2D_MWHI', 'BCUT2D_MWLOW', 'BCUT2D_CHGLO', 'BCUT2D_LOGPHI',
       'BCUT2D_LOGPLOW', 'BCUT2D_MRHI', 'BCUT2D_MRLOW', 'BalabanJ', 'Chi4v',
       'HallKierAlpha', 'Ipc', 'Kappa2', 'Kappa3', 'PEOE_VSA1', 'PEOE_VSA10',
       'PEOE_VSA12', 'PEOE_VSA14', 'PEOE_VSA5', 'PEOE_VSA6', 'PEOE_VSA7',
       'PEOE_VSA8', 'PEOE_VSA9', 'SMR_VSA1', 'SMR_VSA6', 'SlogP_VSA1',
       'SlogP_VSA11', 'SlogP_VSA2', 'SlogP_VSA5', 'EState_VSA10',
       'EState_VSA2', 'EState_VSA3', 'EState_VSA4', 'EState_VSA5',
       'EState_VSA6', 'EState_VSA7', 'EState_VSA8', 'VSA_EState10',
       'VSA_EState3', 'VSA_EState4', 'VSA_EState5', 'VSA_EState7',
       'VSA_EState8', 'VSA_EState9', 'FractionCSP3', 'NumHeteroatoms',
       'MolLogP', 'MolMR', 'fr_allylic_oxid', 'fr_aniline', 'fr_benzene',
       'fr_ester', 'fr_nitro_arom', 'fr_thiazole',"Id2"]

In [42]:
X=train[selected_features]
y=train["Expected"]

Initialized catBoostClassifier and trained the model. Calculated the f1 score for internal validation.

In [43]:
from catboost import CatBoostClassifier
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = CatBoostClassifier(iterations=27000, learning_rate=0.1, depth=6, l2_leaf_reg=3, loss_function='Logloss')
model.fit(X_train, y_train, verbose=False)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: {:.2f}%".format(accuracy*100))
f1= f1_score(y_test, y_pred)
print(f1)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))


Accuracy: 91.26%
0.666497975708502
Accuracy: 0.9125762801804193


Predict the values using test dataset and create submission file using to_csv method.

In [44]:
x=test["x"]
test.drop("x",axis=1,inplace=True)
y_pred=model.predict(test[selected_features])
result = pd.DataFrame({ 'Id': x, 'Predicted': y_pred})
result.to_csv("final10.csv", index=False)
result.head(10)
result.shape

(10994, 2)